# Clasificador - hito 2

In [1]:
import pandas as pd
import numpy as np
import pickle
from string import punctuation
 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
plt.rc('axes', titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams.update({'lines.markeredgewidth': 1})
plt.rcParams.update({'errorbar.capsize': 2})
import plotly
import plotly.express as px

file_names = {
    "df_es_mapping": "../../Data/mapping/df_es_mapping.pickle",
    "df_us_mapping": "../../Data/mapping/df_us_mapping.pickle",
    
    "df_es_test": "../../Data/test/df_es_test.pickle",
    "df_us_test": "../../Data/test/df_us_test.pickle",
    
    "df_es_train": "../../Data/train/df_es_train.pickle",
    "df_us_train": "../../Data/train/df_us_train.pickle",
    
    "df_es_trial": "../../Data/trial/df_es_trial.pickle",
    "df_us_trial": "../../Data/trial/df_us_trial.pickle",
}

# mas imports

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer # tokenizer especial para tweets
tt = TweetTokenizer()
# nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation


**cargar sets**

In [2]:
df_es_train = pickle.load(open(file_names["df_es_train"], "rb"))
df_es_trial = pickle.load(open(file_names["df_es_trial"], "rb"))
df_es_test = pickle.load(open(file_names["df_es_test"], "rb"))

df_us_train = pickle.load(open(file_names["df_us_train"], "rb"))
df_us_trial = pickle.load(open(file_names["df_us_trial"], "rb"))
df_us_test = pickle.load(open(file_names["df_us_test"], "rb"))

**pre-procesamiento**

In [3]:
df_us_train['tokenized_text'] = df_us_train['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))
df_us_train.head()

,id,text,label,tokenized_text
0,729044324441186304,Selfies for summatime @ Drexel University,12,selfies for summatime @ drexel university
1,663834134037442560,Ready to be a bulldog with rasso #hailstate #i...,14,ready to be a bulldog with rasso #hailstate #i...
2,747449193350963200,#scored my new #matcotools #slidehammer weight...,16,#scored my new #matcotools #slidehammer weight...
3,691439672761925637,@user last night was so much fun @ Skyway Thea...,6,@user last night was so much fun @ skyway theatre
4,758118895618109440,love beach days @ Manasquan Beach,12,love beach days @ manasquan beach


In [4]:
df_us_test['tokenized_text'] = df_us_test['text'].str.lower().apply(lambda x: " ".join(tt.tokenize(x)))

In [5]:
"""stopwords_en_withpunct = set(stopwords_en).union(set(punctuation))
print(list(stopwords_en_withpunct)[:10])""";

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(df_us_train["tokenized_text"])
X_test_bow = vectorizer.transform(df_us_test["tokenized_text"])

In [8]:
from sklearn.naive_bayes import MultinomialNB

In [9]:
clf = MultinomialNB()
clf.fit(X_train_bow, df_us_train["label"])

MultinomialNB()

In [10]:
clf.score(X_train_bow, df_us_train["label"])

0.3913558761864433

In [11]:
from sklearn.metrics import classification_report

In [12]:
df_us_mapping = pickle.load(open(file_names["df_us_mapping"], "rb")).sort_values("label")
df_us_mapping

,label,emoji,name
0,0,❤,_red_heart_
1,1,😍,_smiling_face_with_hearteyes_
10,10,📷,_camera_
11,11,🇺🇸,_United_States_
12,12,☀,_sun_
13,13,💜,_purple_heart_
14,14,😉,_winking_face_
15,15,💯,_hundred_points_
16,16,😁,_beaming_face_with_smiling_eyes_
17,17,🎄,_Christmas_tree_


In [13]:
y_pred = clf.predict(X_test_bow)
print(classification_report(df_us_test["label"], y_pred, target_names=df_us_mapping["emoji"]))

/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           ❤       0.26      0.88      0.40     10798
           😍       0.24      0.15      0.18      4830
           📷       0.34      0.05      0.08      1432
          🇺🇸       0.77      0.20      0.32      1949
           ☀       0.42      0.10      0.16      1265
           💜       0.82      0.01      0.02      1114
           😉       0.25      0.00      0.00      1306
           💯       0.51      0.01      0.03      1244
           😁       0.50      0.00      0.01      1153
           🎄       0.80      0.20      0.32      1545
           📸       0.51      0.02      0.03      2417
           😜       0.00      0.00      0.00      1010
           😂       0.30      0.48      0.37      4534
           💕       0.18      0.00      0.01      2605
           🔥       0.59      0.26      0.36      3716
           😊       0.11      0.00      0.00      1613
           😎       0.27      0.02      0.04      1996
           ✨       0.37    

/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
vocab = {k: v for v, k in enumerate(vectorizer.get_feature_names_out())}

In [15]:
vec_test = np.zeros(X_train_bow.shape[1])
k = vocab["santa"]
vec_test[k] = 1
print(vectorizer.inverse_transform([vec_test])[0][0])
clf.predict_proba([vec_test])

santa


array([[0.30575752, 0.12646151, 0.02390757, 0.01609396, 0.04433977,
        0.00681741, 0.02228636, 0.00644249, 0.01469118, 0.07926728,
        0.01555927, 0.01493329, 0.13462598, 0.03244433, 0.0157918 ,
        0.03644817, 0.04698774, 0.0207469 , 0.02034638, 0.01605108]])

### Top palabras por emoji

In [27]:
%%time
vocab_length = X_train_bow.shape[1]
proba_matrix = np.array([clf.predict_proba(np.eye(1,vocab_length,k))[0] for k in range(vocab_length)])

CPU times: user 1h 27min 26s, sys: 28.7 s, total: 1h 27min 55s
Wall time: 14min 40s


In [28]:
print(vocab_length)
print(proba_matrix.shape)

254986
(254986, 20)


In [32]:
una_linea = proba_matrix[:,3]
una_linea.shape

(254986,)

In [37]:
def topPalabras(proba_matrix,emoji_id,k=5):
    # retorna las palabras para las cuales el emoji en cuestión tiene mas probabilidad
    prob = proba_matrix[:,emoji_id]  # mmm
    ind = np.argpartition(prob,-k)[-k:]
    val = prob[ind]
    palabras = [vectorizer.inverse_transform([np.eye(1,vocab_length,k)[0]])[0][0] for k in ind]
    return palabras, val

In [40]:
i = 3
print(df_us_mapping["emoji"][i])
topPalabras(proba_matrix,i)

💕


(['govote', 'imwithher', 'election2016', 'ivoted', 'merica'],
 array([0.74029875, 0.75185152, 0.78014814, 0.82630495, 0.84354542]))

In [42]:
for i in range(20):
    print(df_us_mapping["emoji"][i])
    pal, val = topPalabras(proba_matrix,i)
    print(dict([(pal[j],val[j]) for j in range(len(pal))]))

❤
{'familyfirst': 0.679562931603362, 'heart': 0.6806708579657281, 'valentine': 0.6857972339930336, 'lovemyfamily': 0.685803484186885, 'loveofmylife': 0.7594289068521658}
😍
{'beignet': 0.47919740788782106, 'obsessed': 0.486856749379889, 'swoon': 0.4978084072950244, 'swooning': 0.5325387862637342, 'gorg': 0.5300040945735083}
😂
{'sony': 0.3349669824144724, 'gdlfashion': 0.37337534551598367, 'kae': 0.3859179191618636, 'shredforaliving': 0.39087027038214717, 'bvillain': 0.47826854773146393}
💕
{'govote': 0.7402987474848375, 'imwithher': 0.7518515178366638, 'election2016': 0.7801481398590634, 'ivoted': 0.8263049527529907, 'merica': 0.8435454207090919}
🔥
{'sun': 0.36548880773317577, 'soakin': 0.37805021029035124, 'sunny': 0.38295543509665975, 'beachin': 0.4013070333788539, 'sunshine': 0.4797312481916344}
😊
{'snyder': 0.2833568651329064, 'purplerain': 0.43810939694947926, 'purple': 0.44659009787725723, 'ripprince': 0.2837044945845404, 'endalz': 0.3793652536941223}
😎
{'socratrip': 0.188759073195